# MASONGO UMAR
1800738510         2018/HD05/1967U
## MASTER OF SCIENCE IN COMPUTER SCIENCE
COMPUTER VISION PROJECT EXAM

In [1]:
import cv2
import numpy as np

The whole code in this file enables the creation of a descriptor object that stores 
information about crops (cassava, bananas, etc)features to be extracted


In [3]:
class Descriptor:
    
    # Descriptor class that combines feature descriptors into a single descriptor
    # to produce a feature vector for an input image.
    
    def __init__(self, hog_features=False, hist_features=False,
                     spatial_features=False, hog_lib="cv", size=(64,64),
                     hog_bins=9, pix_per_cell=(8,8), cells_per_block=(2,2),
                     block_stride=None, block_norm="L1", transform_sqrt=True,
                     signed_gradient=False, hist_bins=16, spatial_size=(16,16)):
            
        # Set feature parameters for HOG using either the OpenCV implementation 
        # cv2.HOGDescriptor or scikit-image implementation skimage.feature.hog implementation hog_lib
        
        self.hog_features = hog_features
        self.hist_features = hist_features
        self.spatial_features = spatial_features
        self.size = size
        self.hog_lib = hog_lib
        self.pix_per_cell = pix_per_cell
        self.cells_per_block = cells_per_block
        
        winSize = size
        cellSize = pix_per_cell
        blockSize = (cells_per_block[0] * cellSize[0],
                         cells_per_block[1] * cellSize[1])
            
        if block_stride is not None:
            blockStride = self.block_stride
        else:
            blockStride = (int(blockSize[0] / 2), int(blockSize[1] / 2))

        nbins = hog_bins
        derivAperture = 1
        winSigma = -1.
        histogramNormType = 0
        L2HysThreshold = 0.2
        gammaCorrection = 1
        nlevels = 64
        signedGradients = signed_gradient

        self.HOGDescriptor = cv2.HOGDescriptor(winSize, blockSize,
                    blockStride, cellSize, nbins, derivAperture, winSigma,
                    histogramNormType, L2HysThreshold, gammaCorrection,
                    nlevels, signedGradients)
        
    def getFeatureVector(self, image):

        # Return the feature vector for an image

        if image.shape[:2] != self.size:
            image = cv2.resize(image, self.size, interpolation=cv2.INTER_AREA)

        feature_vector = np.array([])

        if self.hog_features:
            feature_vector  = np.hstack(
                (feature_vector, self.HOGDescriptor.compute(image)[:,]))

        if self.hist_features:
            # np.histogram() returns a tuple if given a 2D array and an array
            # if given a 3D array. 
            # To maintain compatibility with other functions in the object detection pipeline,
            # check that the input array has three dimensions. Add axis if necessary.
            # Note that histogram bin range assumes uint8 array.
            if len(image.shape) < 3:
                image = image[:, :, np.newaxis]

            hist_vector = np.array([])
            for channel in range(image.shape[2]):
                channel_hist = np.histogram(image[:, :, channel],
                        bins=self.hist_bins, range=(0, 255))[0]
                hist_vector = np.hstack((hist_vector, channel_hist))
            feature_vector = np.hstack((feature_vector, hist_vector))

        if self.spatial_features:
            spatial_image = cv2.resize(image, self.spatial_size,
                    interpolation=cv2.INTER_AREA)
            spatial_vector = spatial_image.ravel()
            feature_vector = np.hstack((feature_vector, spatial_vector))
        
        return feature_vector